# LAB2

Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, November 26 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices,randint
import random

import lab2_lib

In [2]:
from random import choices, randint, sample

class EvolutionaryAlgorithm:
    def __init__(self, problem, population_size, generations, mutation_rate,genome_length):
        self.problem = problem
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.genome_lenght = genome_length

# Initialize populations of genome_lenght genomes
    def initialize_population(self):
        return [choices([0, 1], k=self.genome_lenght) for _ in range(self.population_size)]

# Mutation of 1/5 of the genome as a bit-flipping 
    def mutate(self, individual):
        num_mutations = self.genome_lenght // 5  # 
        for _ in range(num_mutations):
            mutated_index = randint(0, len(individual) - 1)
            individual[mutated_index] = 1 - individual[mutated_index]
        return individual


# Crossover between two parent 
    def crossover(self, parent1, parent2):
        crossover_point = randint(1, self.genome_lenght-1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    
# Evaluate current population
    def evaluate_population(self,population):
        return [self.problem(individual) for individual in population]
        

# Tournament selection between 5 individuals
    def tournament_selection(self, population, fitnesses, tournament_size=5):
        extracted = []
        for _ in range(2):
            tournament_indices = choices(range(self.population_size), k=tournament_size)
            tournament = [(population[i], fitnesses[i]) for i in tournament_indices]
            winner = max(tournament, key=lambda x: x[1])[0]
            extracted.append(winner)
        return extracted

    def run(self):
        population = self.initialize_population()
        offspring = []
        best_fitness = []

        for generation in range(self.generations):
            # Evaluate fitness for each individual in the population
            fitness_values = [self.problem(individual) for individual in population]

            # Generates offspring
            for _ in range(self.population_size //2):
                parent1, parent2 = self.tournament_selection(population, fitness_values)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                offspring.extend([child1, child2])


            # Add to the old population the offspring
            population += offspring
            best_fitness.append(max(fitness_values))
        
            # Checking convergence 
            if generation > 0 and abs(max(fitness_values) - best_fitness[-2]) < 0.01:
                print(f"Convergence reached at generation {generation}.")
                break

        # Return the best individual found
        best_individual = max(population, key=lambda ind: self.problem(ind))
        return best_individual, max(best_fitness)


In [3]:
# Example:
problem_size = 1
fitness = lab2_lib.make_problem(problem_size)
ea = EvolutionaryAlgorithm(fitness, population_size=100, generations=1000, mutation_rate=0.1, genome_length=1000)
best_individual,best_fitness = ea.run()
#print("Best Individual:", best_individual)
print("Best Fitness:", best_fitness)
print("Number of Fitness Calls:", fitness.calls)

Convergence reached at generation 1.
Best Fitness: 0.548
Number of Fitness Calls: 700
